# Komplettes Skript

In [4]:
# mini_cifar10_cnn.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ---- Setup ----
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
torch.manual_seed(0)

# CIFAR-10: 32x32 RGB, Klassen=10
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2470, 0.2435, 0.2616)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2470, 0.2435, 0.2616)),
])

train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_ds  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=1, pin_memory=True)

# ---- Kleines CNN ----
class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)   # 32x32 -> 32x32
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 32x32 -> 32x32
        self.pool  = nn.MaxPool2d(2, 2)               # 32x32 -> 16x16
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # 16x16 -> 16x16
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)# 16x16 -> 16x16
        self.head  = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),                  # -> 128x1x1
            nn.Flatten(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.head(x)
        return x

model = SmallCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 15

# ---- Training ----
for epoch in range(1, epochs + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # ---- Evaluation ----
    model.eval()
    correct, total, test_loss = 0, 0, 0.0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            loss = criterion(logits, labels)
            test_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_loss /= total
    test_acc = correct / total
    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} acc={train_acc:.3f} | "
          f"test_loss={test_loss:.4f} acc={test_acc:.3f}")

# Optional: Modell speichern
torch.save(model.state_dict(), "small_cifar10_cnn.pt")
print("Saved to small_cifar10_cnn.pt")


Epoch 01 | train_loss=1.8255 acc=0.298 | test_loss=1.6096 acc=0.384
Epoch 02 | train_loss=1.5282 acc=0.428 | test_loss=1.4053 acc=0.485
Epoch 03 | train_loss=1.3554 acc=0.507 | test_loss=1.2984 acc=0.540
Epoch 04 | train_loss=1.2186 acc=0.560 | test_loss=1.1459 acc=0.584
Epoch 05 | train_loss=1.1229 acc=0.598 | test_loss=1.1268 acc=0.602
Epoch 06 | train_loss=1.0619 acc=0.620 | test_loss=1.0496 acc=0.629
Epoch 07 | train_loss=1.0128 acc=0.638 | test_loss=1.0087 acc=0.642
Epoch 08 | train_loss=0.9681 acc=0.656 | test_loss=1.0544 acc=0.624
Epoch 09 | train_loss=0.9297 acc=0.672 | test_loss=0.9786 acc=0.652
Epoch 10 | train_loss=0.8881 acc=0.684 | test_loss=0.8811 acc=0.689
Epoch 11 | train_loss=0.8521 acc=0.698 | test_loss=0.8724 acc=0.691
Epoch 12 | train_loss=0.8185 acc=0.712 | test_loss=0.9134 acc=0.682
Epoch 13 | train_loss=0.8019 acc=0.717 | test_loss=0.8620 acc=0.702
Epoch 14 | train_loss=0.7662 acc=0.730 | test_loss=0.8321 acc=0.717
Epoch 15 | train_loss=0.7409 acc=0.739 | test_lo